### Visualization

In [1]:
import numpy as np
import plotly
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import pandas as pd
import numpy as np

plotly.offline.init_notebook_mode(connected=True)

In [2]:
init_notebook_mode(connected=True)

def process_train_logs(path):
    with open(path) as f:
        lines = f.readlines()
    frames = []
    spl = []
    success = []
    for line in lines:
        if "frames: " in line:
            frames.append(int(line.strip().split("frames: ")[-1]))
        elif "spl:" in line or "loop_spl: " in line:
            cols = line.strip().split()
            spl.append(float(cols[-3]))
            success.append(float(cols[-1]))
    n = min(len(frames), len(spl))
    frames = frames[:n]
    spl, success = spl[:n], success[:n]
    return {
        "frames": frames,
        "spl": spl,
        "success": success
    }

In [3]:
BP = "/checkpoint/akadian/exp-dir/"

log_file_paths = [
    (BP + "job_12337109.exp_habitat_api_navigation_analysis_datetime_2019_04_29_08_47_33/train.log", "pointnav-blind"),
    (BP + "job_12344103.exp_habitat_api_navigation_analysis_datetime_2019_04_29_11_01_19/train.log", "loopnav-blind"),
    (BP + "job_12337185.exp_habitat_api_navigation_analysis_datetime_2019_04_29_08_48_44/train.log", "depth-pointnav"),
    (BP + "job_12337152.exp_habitat_api_navigation_analysis_datetime_2019_04_29_08_48_09/train.log", "depth-loopnav"),
]

In [4]:
train_results = [(process_train_logs(p[0]), p[1]) for p in log_file_paths]

spl_data = []
for tr in train_results:
    t = go.Scatter(x=tr[0]["frames"], y=tr[0]["spl"], name=tr[1])
    spl_data.append(t)
    
layout = dict(
    title="SPL training curve",
    xaxis=dict(title="Frames"),
    yaxis=dict(range=(0, 1.1))
)

fig = dict(data=spl_data, layout=layout)
iplot(fig, show_link=False)

In [5]:
success_data = []
for tr in train_results:
    t = go.Scatter(x=tr[0]["frames"], y=tr[0] ["success"], name=tr[1])
    success_data.append(t)
    
layout = dict(
    title="Success training curve",
    xaxis=dict(title="Frames"),
    yaxis=dict(range=(0, 1.1))
)

fig = dict(data=success_data, layout=layout)
iplot(fig, show_link=False)

In [6]:
with open("/private/home/akadian/navigation-analysis/navigation-analysis-habitat/evaluate.blind.loopnav.T_S.log") as f:
    lines = f.readlines()
    
valid_lines = [line for line in lines if "T SPL:" in line]

s1_spl = []

for line in valid_lines:
    s1_spl.append(float(line.strip().split()[-1]))

In [7]:
with open("/private/home/akadian/navigation-analysis/navigation-analysis-habitat/evaluate.blind.loopnav.S_T_S.log") as f:
    lines = f.readlines()

valid_lines = [line for line in lines if "S1 SPL:" in line]

stemp_spl = []
s2_spl = []

for line in valid_lines:
    stemp_spl.append(float(line.strip().split(",")[-3].split()[-1]))
    s2_spl.append(float(line.strip().split(",")[-2].split()[-1]))
    
print("Stage-1 SPL: {:.3f}\nStage-2 SPL: {:.3f}".format(np.mean(s1_spl), np.mean(s2_spl)))


Stage-1 SPL: 0.616
Stage-2 SPL: 0.912


In [8]:
hist1 = go.Histogram(
    x=s1_spl,
    opacity=0.75,
    name="Stage-1",
)

hist2 = go.Histogram(
    x=s2_spl,
    opacity=0.75,
    name="Stage-2",
)
hist3 = go.Histogram(
    x=stemp_spl,
    opacity=0.75,
    name="Stage-Temp"
)

data = [hist1, hist2]
layout = go.Layout(
    barmode='overlay', 
    title="LoopNav", 
    xaxis=dict(title="SPL"),
    shapes=[{
        "line": {
                    "color": "#0099FF", 
                    "dash": "solid", 
                    "width": 1
                },
        "type": "line",
        "x0": np.mean(s1_spl),
        "x1": np.mean(s1_spl),
        "xref": "x",
        "y0": 0,
        "y1": 0.98,
        "yref": "paper"
        }, {
        "line": {
                    "color": "#FDAB5A", 
                    "dash": "solid", 
                    "width": 1
                },
        "type": "line",
        "x0": np.mean(s2_spl),
        "x1": np.mean(s2_spl),
        "xref": "x",
        "y0": 0,
        "y1": 0.98,
        "yref": "paper"
        }
    ],
    annotations=[
        dict(
            x=np.mean(s1_spl),
            y=1,
            xref="x",
            yref="paper",
            text="S1 = {:.3f}".format(np.mean(s1_spl)),
            showarrow=True,
            arrowhead=7,
            ax=1,
            ay=1,
        ),
        dict(
            x=np.mean(s2_spl),
            y=1,
            xref="x",
            yref="paper",
            text="S2 = {:.3f}".format(np.mean(s2_spl)),
            showarrow=True,
            arrowhead=7,
            ax=1,
            ay=1,
        )
    ]
)
fig = go.Figure(data=data, layout=layout)
iplot(fig, show_link=False)

In [10]:
scatter = go.Scatter(
    x=s1_spl,
    y=s2_spl,
    mode="markers",
)

data = [scatter]
layout = dict(
    title="LoopNav Scatter Plot",
    xaxis=dict(title="Stage-1 SPL", range=(-0.05, 1.05)),
    yaxis=dict(title="Stage-2 SPL", range=(-0.05, 1.05)),
    shapes=[{
        "line": {
                    "color": "black", 
                    "dash": "dash", 
                    "width": 1
                },
        "type": "line",
        "x0": 0,
        "x1": 1,
        "xref": "x",
        "y0": 0.05,
        "y1": 1 / 1.05,
        "yref": "paper"
        }
    ]
)

fig = go.Figure(data=data, layout=layout)
iplot(fig, show_link=False)

In [2]:
with open("/private/home/akadian/navigation-analysis/navigation-analysis-habitat/evaluate.blind.loopnav.S_T_S.log") as f:
    lines = f.readlines()

In [3]:
episode_start_inds = []

for i in range(len(lines)):
    if "CURRENT_EPISODE" in lines[i]:
        episode_start_inds.append(i)

In [4]:
def eval_pos(line):
    cols = line.strip().split("[")
    cols = cols[-1][:-1].split()
    cols = [float(x) for x in cols]
    return cols

def eval_rot(line):
    cols = line.strip().split()
    res = [
        float(cols[-4].split("quaternion(")[-1][:-1]),
        float(cols[-3][:-1]),
        float(cols[-2][:-1]),
        float(cols[-1][:-1])
    ]
    return res

In [5]:
n = 1000
sts_res = []

for i in range(n):
    ind1 = episode_start_inds[i]
    ind2 = episode_start_inds[i + 1]
    
    r = dict(
        start_position=None,
        start_rotation=None,
        s1_ending_position=None,
        s1_ending_rotation=None,
        s2_ending_position=None,
        s2_ending_rotation=None,
    )
    
    for line in lines[ind1:ind2]:
        if "START_POSITION A" in line:
            r["start_position"] = eval_pos(line)
        elif "START_ROTATION A" in line:
            rot = eval_rot(line)
            r["start_rotation"] = [rot[1], rot[2], rot[3], rot[0]]
        elif "STAGE-1 ENDING_POSITION" in line:
            r["s1_ending_position"] = eval_pos(line)
        elif "STAGE-1 ENDING_ROTATION" in line:
            rot = eval_rot(line)
            r["s1_ending_rotation"] = [rot[1], rot[2], rot[3], rot[0]]
        elif "STAGE-2 ENDING_POSITION" in line:
            r["s2_ending_position"] = eval_pos(line)
        elif "STAGE-2 ENDING_ROTATION" in line:
            rot = eval_rot(line)
            r["s2_ending_rotation"] = [rot[1], rot[2], rot[3], rot[0]]
            
    sts_res.append(r)

In [6]:
sts_success = []

for line in lines:
    if "T SPL:" in line:
        sts_success.append(float(line.strip().split()[-1]) > 0)

In [7]:
len(sts_success), len(sts_res)

(1000, 1000)

In [8]:
assert len(sts_success) == len(sts_res)

In [9]:
for i in range(n):
    sts_res[i]["success"] = sts_success[i]

In [10]:
sts_res[0]

{'start_position': [-0.35736826, 0.1582519, 0.8391239],
 'start_rotation': [0.0, 0.979339718818665, 0.0, -0.202222123742104],
 's1_ending_position': [-10.9828005, 0.1582519, -4.8579416],
 's1_ending_rotation': [0.0, -0.144927173852921, 0.0, 0.989442408084869],
 's2_ending_position': [-0.33614418, 0.1582519, 0.91141224],
 's2_ending_rotation': [0.0, 0.993237853050232, 0.0, -0.11609759926796],
 'success': True}

In [11]:
import pickle

with open("/private/home/akadian/navigation-analysis/navigation-analysis-habitat/sts_episodes.pkl", "wb") as f:
    pickle.dump(sts_res, f)

In [17]:
ls -lh -t /private/home/akadian/navigation-analysis/navigation-analysis-habitat/eval-videos-loopnav

total 87M
-rw-rw-r-- 1 akadian akadian 6.2M May  7 12:19 10_apt_spl1_0.62_spl2_0.95.mp4
-rw-rw-r-- 1 akadian akadian 6.9M May  7 12:18 9_apt_spl1_0.86_spl2_0.97.mp4
-rw-rw-r-- 1 akadian akadian 4.1M May  7 12:17 8_apt_spl1_0.72_spl2_1.00.mp4
-rw-rw-r-- 1 akadian akadian 4.5M May  7 12:16 7_apt_spl1_0.72_spl2_0.98.mp4
-rw-rw-r-- 1 akadian akadian  12M May  7 12:15 6_apt_spl1_0.20_spl2_0.89.mp4
-rw-rw-r-- 1 akadian akadian 3.4M May  7 12:13 5_apt_spl1_0.91_spl2_1.00.mp4
-rw-rw-r-- 1 akadian akadian 3.7M May  7 12:13 4_apt_spl1_0.98_spl2_0.99.mp4
-rw-rw-r-- 1 akadian akadian 5.9M May  7 12:12 3_apt_spl1_0.90_spl2_1.00.mp4
-rw-rw-r-- 1 akadian akadian  20M May  7 12:11 2_apt_spl1_0.17_spl2_0.94.mp4
-rw-rw-r-- 1 akadian akadian 4.5M May  7 12:06 1_apt_spl1_0.77_spl2_0.97.mp4
-rw-rw-r-- 1 akadian akadian  18M May  7 12:06 0_apt_spl1_0.30_spl2_0.73.mp4


In [18]:
ls -lh -t /private/home/akadian/navigation-analysis/navigation-analysis-habitat/eval-videos-pointnav

total 93M
-rw-rw-r-- 1 akadian akadian 2.0M May  7 12:17 17_apt_spl_0.89.mp4
-rw-rw-r-- 1 akadian akadian 8.1M May  7 12:17 16_apt_spl_0.41.mp4
-rw-rw-r-- 1 akadian akadian 2.6M May  7 12:15 15_apt_spl_0.45.mp4
-rw-rw-r-- 1 akadian akadian 7.4M May  7 12:15 14_apt_spl_0.35.mp4
-rw-rw-r-- 1 akadian akadian 3.6M May  7 12:13 13_apt_spl_0.74.mp4
-rw-rw-r-- 1 akadian akadian 7.8M May  7 12:13 12_apt_spl_0.56.mp4
-rw-rw-r-- 1 akadian akadian 4.7M May  7 12:11 11_apt_spl_0.37.mp4
-rw-rw-r-- 1 akadian akadian 1.8M May  7 12:10 10_apt_spl_0.92.mp4
-rw-rw-r-- 1 akadian akadian 6.3M May  7 12:10 9_apt_spl_0.44.mp4
-rw-rw-r-- 1 akadian akadian 6.8M May  7 12:09 8_apt_spl_0.27.mp4
-rw-rw-r-- 1 akadian akadian 3.6M May  7 12:08 7_apt_spl_0.64.mp4
-rw-rw-r-- 1 akadian akadian 2.3M May  7 12:07 6_apt_spl_0.80.mp4
-rw-rw-r-- 1 akadian akadian 1.8M May  7 12:07 5_apt_spl_0.91.mp4
-rw-rw-r-- 1 akadian akadian 3.1M May  7 12:06 4_apt_spl_0.66.mp4
-rw-rw-r-- 1 akadian akadian 5.4M May  7 12:06 3_apt_spl_0